In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 3060 Laptop GPU


In [2]:
import os
import glob
import cv2
import numpy as np
from tqdm import tqdm
from facenet_pytorch import MTCNN
import torch
from PIL import Image

def create_face_video(input_video_path, output_video_path, max_frames=150):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Initialize MTCNN face detector
    mtcnn = MTCNN(keep_all=True, device=device)

    vid_obj = cv2.VideoCapture(input_video_path)
    frame_count = int(vid_obj.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = vid_obj.get(cv2.CAP_PROP_FPS) or 30  # Default to 30 if fps can't be read
    print(f"Processing video: {input_video_path}")
    print(f"Total frames in video: {frame_count}, FPS: {fps}")

    # Prepare video writer
    height, width = 112, 112
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    frame_idx = 0
    success = True
    # pbar = tqdm(total=min(frame_count, max_frames), desc="Processing frames")
    while success and frame_idx < max_frames:
        success, frame = vid_obj.read()
        if not success:
            print(f"Failed to read frame {frame_idx}")
            break

        try:
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(rgb_frame)

            # Detect faces
            boxes, _ = mtcnn.detect(img)

            if boxes is None:
                print(f"No faces detected in frame {frame_idx}")
                frame_idx += 1
                # pbar.update(1)
                continue

            # Loop through detected faces
            for box in boxes:
                left, top, right, bottom = [int(b) for b in box]
                # Ensure coordinates are within image bounds
                top = max(0, top)
                left = max(0, left)
                bottom = min(frame.shape[0], bottom)
                right = min(frame.shape[1], right)

                face_image = frame[top:bottom, left:right]
                if face_image.size == 0:
                    print(f"Empty face detected in frame {frame_idx}")
                    continue

                # Resize the face to the desired size
                face_resized = cv2.resize(face_image, (width, height))
                out.write(face_resized)

        except Exception as e:
            print(f"Error processing frame {frame_idx}: {e}")

        frame_idx += 1
        # pbar.update(1)

    # pbar.close()
    vid_obj.release()
    out.release()

In [3]:
# Main function to process an entire dataset
def preprocess_dataset(video_dir, output_dir, max_frames=150):
    os.makedirs(output_dir, exist_ok=True)
    video_files = glob.glob(os.path.join(video_dir, '*.mp4'))
    i = 0
    n = len(video_files)
    print(f"Found {n} videos to process.")
    for video_path in tqdm(video_files, desc="Processing videos"):
        video_name = os.path.basename(video_path)
        output_video_path = os.path.join(output_dir, video_name)
        i+=1
        print(f"{i}/{n}")
        # Skip if output video already exists
        if os.path.exists(output_video_path):
            print(f"File already exists: {output_video_path}")
            continue
       
        # Process and save face-cropped video
        create_face_video(video_path, output_video_path, max_frames=max_frames)
        

In [5]:
input_video_dir = r'C:\Users\KK\Desktop\dfd\dfdc_train_part_5'  
output_video_dir = r'C:\Users\KK\Desktop\dfd\data\CroppedFaces'  
preprocess_dataset(input_video_dir, output_video_dir)

Found 1334 videos to process.


Processing videos:   0%|                                                                      | 0/1334 [00:00<?, ?it/s]

1/1334
File already exists: C:\Users\KK\Desktop\dfd\processed\aaqaifqrwn.mp4
2/1334
File already exists: C:\Users\KK\Desktop\dfd\processed\aayrffkzxn.mp4
3/1334
File already exists: C:\Users\KK\Desktop\dfd\processed\abhggqdift.mp4
4/1334
File already exists: C:\Users\KK\Desktop\dfd\processed\acagallncj.mp4
5/1334
File already exists: C:\Users\KK\Desktop\dfd\processed\acdkfksyev.mp4
6/1334
File already exists: C:\Users\KK\Desktop\dfd\processed\achdeirhym.mp4
7/1334
File already exists: C:\Users\KK\Desktop\dfd\processed\acryukkddn.mp4
8/1334
File already exists: C:\Users\KK\Desktop\dfd\processed\adftmlvzfs.mp4
9/1334
File already exists: C:\Users\KK\Desktop\dfd\processed\aegvtnwqoe.mp4
10/1334
File already exists: C:\Users\KK\Desktop\dfd\processed\aejvkfbtxs.mp4
11/1334
File already exists: C:\Users\KK\Desktop\dfd\processed\aeqpxjlbwu.mp4
12/1334
File already exists: C:\Users\KK\Desktop\dfd\processed\ahhdprhoww.mp4
13/1334
File already exists: C:\Users\KK\Desktop\dfd\processed\aimzesksew

C:\Users\KK\AppData\Roaming\Python\Python312\site-packages\facenet_pytorch\models\mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_di

Processing video: C:\Users\KK\Desktop\dfd\dfdc_train_part_0\fxredodlkl.mp4
Total frames in video: 300, FPS: 29.9688


...

No faces detected in frame 149
368/1334
Using device: cuda
Processing video: C:\Users\KK\Desktop\dfd\dfdc_train_part_0\hnalkvbcim.mp4
Total frames in video: 300, FPS: 29.969


Processing videos:  28%|████████████████                                          | 368/1334 [57:07<7:25:37, 27.68s/it]

369/1334
Using device: cuda
Processing video: C:\Users\KK\Desktop\dfd\dfdc_train_part_0\horutnradk.mp4
Total frames in video: 300, FPS: 29.969


Processing videos:  28%|████████████████                                          | 368/1334 [57:34<2:31:09,  9.39s/it]


KeyboardInterrupt: 